In [ ]:
    import sys

    sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
    #sys.path.append("C:/Users/jaesu/AppData/Local/Programs/Python/Python312/Lib/site-packages")
    sys.path.append("/scratch/group/csce435-f24/thicket")
    from glob import glob

    import matplotlib.pyplot as plt 
    import pandas as pd

    import thicket as th

    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)

In [ ]:
tk = th.Thicket.from_caliperreader(glob("cali_files/sample_cali/*.cali"))

In [ ]:
tk.metadata_column_to_perfdata("algorithm")
tk.metadata_column_to_perfdata("input_size")
tk.metadata_column_to_perfdata("input_type")
tk.metadata_column_to_perfdata("num_procs")

tk.dataframe = tk.dataframe.reset_index().set_index(["node", "algorithm", "input_size", "input_type", "num_procs"]).sort_index()

tk.dataframe

In [6]:
# Define common variables
processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
input_size = 67108864
input_types = ["sorted", "random", "reverse", "perturbed"]

In [11]:
plt.rcParams.update({"font.size": 18})

In [ ]:
# Check unique values in the algorithm metadata to see the correct naming
print(tk.dataframe.index.get_level_values('algorithm').unique())

In [ ]:
alg = "sample"

# Dictionary to store speedup data for each input type
speedup_data_all_types = {input_type: {} for input_type in input_types}

for input_type in input_types:
    
    # Get the sequential (1 processor) Avg time/rank for the current input type
    seq_time = tk.dataframe.loc[(slice(None), alg, input_size, input_type, 1), "Avg time/rank"].values[0]
        
    # Calculate speedup for each processor count
    for proc in processes:
        
        # Fetch the parallel time for the current processor count and input type
        parallel_time = tk.dataframe.loc[(slice(None), alg.lower(), input_size, input_type, proc), "Avg time/rank"].values[0]
        speedup_data_all_types[input_type][proc] = seq_time / parallel_time

plt.figure(figsize=(15, 7))
for input_type, speedup_data in speedup_data_all_types.items():
    plt.plot(list(speedup_data.keys()), list(speedup_data.values()), marker='o', label=input_type.capitalize())

# This is to fix bitonic sort's title 
if alg == "bitonic_sort":
    alg = "Bitonic"

plt.title(f"{alg} Sort - Strong Scaling Speedup (Input Size: {input_size})")
plt.xlabel("Number of Processes")
plt.ylabel("Speedup (Sequential Time / Parallel Time)")
plt.grid(True)
plt.legend(title="Input Type")
plt.savefig(f"./graphs/{alg}_speedup_{input_size}.png")
plt.show()
